In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from scipy.constants import N_A
from scipy.constants import physical_constants
import numpy as np 
from tools.component_tools import Component
from tools.component_tools import Fluid
from tools.component_tools import Membrane
from tools.component_tools import GLC_Gas
from tools.component_tools import GLC 
from tools.materials import Flibe 
import tools.correlations as corr
import tools.materials as materials
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
from scipy.optimize import fsolve
import matplotlib.lines as mlines
from tools.component_tools import BreedingBlanket
from scipy.constants import R
T=973.15
d_hyd=25.4E-3
U0=2.5
L=10
flibe=Fluid(d_Hyd=d_hyd,U0=U0)
flibe.set_properties_from_fluid_material(materials.Flibe(T))
c_in=1E-3
Steel = Membrane( thick=0.25E-3)
Steel.set_properties_from_solid_material(materials.Steel(T))
PAV = Component(c_in=c_in, fluid=flibe, membrane=Steel)   
PAV.use_analytical_efficiency(L)
PAV.outlet_c_comp()
PAV.inspect(["c_in"])


c_in: 0.001


In [ ]:


Q=0.625E9
TBR=1.08
eV_to_J = physical_constants["electron volt-joule relationship"][0]
reaction_energy = 17.6e6  # reaction energy in eV 17.6 MeV
neutrons = Q / (reaction_energy * eV_to_J)

tritium_gen = TBR * neutrons / N_A 
recirculation_vec=np.linspace(0,10,11)
c_out_PAV_vec=np.array([])
eff_vec=np.array([])
extract_vec=np.array([])
n_pipes_vec=np.array([])
T=973.15
L=10
d_hyd=25.4E-3
U0=2.5
L=6
c0=1E-3
mat=materials.Flibe(T)
rho_flibe=2415.6-0.49072*T
mu_flibe=1.16E-4*np.exp(3755/T)
k_H=8.27E-9*np.exp(4.27E-3*T)
D_flibe=9.3E-7*np.exp((-42E3)/(R*T))
K_S_S=953*np.exp((-10.7E3)/(R*T))*1E-3
D_S=7.43E-7*np.exp((-44.1E3)/(R*T))
for recirculation in recirculation_vec:
    
    err=1
    tol=1E-6
    c_in=c0
    i=0
    # flibe=Fluid(T=T, Solubility=k_H, MS=True,D=D_flibe, d_Hyd=d_hyd ,mu=mu_flibe,rho=rho_flibe,U0=U0*(recirculation+1),k=mat.k,
    #     cp=mat.cp)
    flibe=Fluid(d_Hyd=d_hyd,rho=rho_flibe,U0=U0*(recirculation+1))
    flibe.set_properties_from_fluid_material(mat)
    Steel = Membrane( T=T,
            D=D_S,
            thick=0.25E-3,
            K_S=K_S_S,
            k_d=1E6,
            k_r=1E6,k=21)
    PAV = Component(c_in=c_in, fluid=flibe, membrane=Steel)   
    while err>tol:
        i=i+1
        c_in1=c_in
        PAV.update_attribute("c_in",c_in)
        PAV.analytical_efficiency(L)
        PAV.update_attribute("eff",PAV.eff_an)
        PAV.outlet_c_comp()
        c_in=(PAV.c_out*recirculation+c0)/(recirculation+1)
        err=abs((c_in-c_in1)/c_in)
    c_out_PAV_vec=np.append(c_out_PAV_vec,PAV.c_out)
    eff_vec=np.append(eff_vec,PAV.eff_an)
    extract_vec=np.append(extract_vec,(PAV.c_in-PAV.c_out)*PAV.fluid.U0*PAV.fluid.d_Hyd**2/4)
    n_pipes_vec=np.append(n_pipes_vec,tritium_gen/((PAV.c_in-PAV.c_out)*PAV.fluid.U0*PAV.fluid.d_Hyd**2/4*np.pi))
plt.plot(recirculation_vec,eff_vec)
plt.xlabel("Recirculation ratio")
plt.ylabel("Extraction Efficiency [%]")
plt.show()